# --- Day 11: Radioisotope Thermoelectric Generators ---

http://adventofcode.com/2016/day/11

---

Four floors, we need to know the min num of moves to move all items to the fourth floor, with constraints being:

- two types of items, RTGs and Chips, both of which have a type
- an elevator which can carry two items, and only works if it contains at least one
- chips are damanged if on same floor as another kind of RTGs
- chips are safe if coupled with their RTG

In [7]:
with open(f'inputs/11.txt') as f:
    data = f.read().strip().splitlines()
data

['The first floor contains a polonium generator, a thulium generator, a thulium-compatible microchip, a promethium generator, a ruthenium generator, a ruthenium-compatible microchip, a cobalt generator, and a cobalt-compatible microchip.',
 'The second floor contains a polonium-compatible microchip and a promethium-compatible microchip.',
 'The third floor contains nothing relevant.',
 'The fourth floor contains nothing relevant.']

I started making a class which contains what the state floors and elevators is and methods to move things, but it seemed way too much code then just using namedtuple. So the following state tuple captures all the information of our "world" at a given point in time:

In [8]:
from collections import namedtuple, defaultdict, deque

state = namedtuple("State", ["elavator", "steps", "floors"])

one = ("POG", "THG", "THM", "PRG", "RUG", "RUM", "COG", "COM")
two = ("POM", "PRM")
three, four = (), ()

#start_state = state(1, {1: one, 2: two, 3:three, 4:four}
start_state = state(elavator=0,steps=0, floors=(one, two, three, four))
start_state

State(elavator=0, steps=0, floors=(('POG', 'THG', 'THM', 'PRG', 'RUG', 'RUM', 'COG', 'COM'), ('POM', 'PRM'), (), ()))

the test state:

In [9]:
test_floors = (("HYM", "LIM"), ("HYG",), ("LIG",), ())
test_state = state(0, 0, test_floors)
test_state

State(elavator=0, steps=0, floors=(('HYM', 'LIM'), ('HYG',), ('LIG',), ()))

Since I want to [build a graph using a dict](https://github.com/khalido/adventofcode2017/blob/master/Day%2012%20-%20Digital%20Plumber.ipynb), I want to make sure I can use this in a dict:

In [10]:
d = defaultdict(list)
d[start_state] = [start_state, start_state]
d

defaultdict(list,
            {State(elavator=0, steps=0, floors=(('POG', 'THG', 'THM', 'PRG', 'RUG', 'RUM', 'COG', 'COM'), ('POM', 'PRM'), (), ())): [State(elavator=0, steps=0, floors=(('POG', 'THG', 'THM', 'PRG', 'RUG', 'RUM', 'COG', 'COM'), ('POM', 'PRM'), (), ())),
              State(elavator=0, steps=0, floors=(('POG', 'THG', 'THM', 'PRG', 'RUG', 'RUM', 'COG', 'COM'), ('POM', 'PRM'), (), ()))]})

I had to modify the state function so it used immutable objects otherwise I couldn't use it as a dict key.

So now I need a function which keeps moving objects until they all are on the fourth floor. There are must be many difffernt ways to move everything to the top, so this is a path problem - I have a start state, from which I can move to many diffeerent possible states, and so on, with some of them ending in the solution, so I need a way to build out a graph and find the shortest path to our end goal.

First up helper functions:

In [11]:
def reached_goal(st):
    """returns True if state is at the goal of everything being at the fourth floor"""
    if not (st.floors[0] or st.floors[1] or st.floors[2]):
        return True
    else:
        return False

solved_state = state(3, 4, ((), (), (), (1,2)))
reached_goal(start_state), reached_goal(solved_state)

(False, True)

Now to check if a given floor or state is valid:
**make legal_floor func faster, doesn't look optimal atm**

In [12]:
def is_legal_floor(floor):
    """takes in floor, returns True if valid, false otherwise
    Note: refactor to make this faster"""
    gens = {i[:2] for i in floor if i[-1] == "G"}
    chips = {i[:2] for i in floor if i[-1] == "M"}
    
    # check that every chip has a corresponding gen
    for chip in chips:
        if chip not in gens and len(gens) > 0:
            return False 
    return True
        
def is_valid(state):
    """takes in a state and returns True if valid, false otherwise
    determines validity to be if no chips have been fried"""
    for floor_num, floor in enumerate(state.floors):
        if not is_legal_floor(floor):
            return False
    return True
        
invalid_state = state(1,0, (("POM", "PRM", "RUG"), two, three, four))
is_valid(start_state), is_valid(invalid_state)

(True, False)

Now to generate a graph of states, one leading to the next. The elavator has to move at least one item, with a max of two, so one way to generate all the possible moves is to generate a list of all possible combinations of items on a floor which can be moved:

In [13]:
from itertools import combinations, chain

def filter_pairs(pairs):
    """takes in a list of pairs and removes equivalent pairs
    since all matching chip/gen pairs are the same for this problem"""
    
    equivalent_pairs = [p for p in pairs if p[0][:-1]==p[1][:-1]]
    if len(equivalent_pairs) > 1:
        #keep first one, delete others
        return [p for p in pairs if p not in equivalent_pairs[1:]]
        
    return pairs

def get_combos(floor):
    """returns combinations of paris, filtering for equivalent pairs"""
    pairs = filter_pairs([i for i in combinations(floor, 2)])
    singles = [i for i in combinations(floor, 1)]
    
    return pairs + singles

get_combos(start_state.floors[0])

[('POG', 'THG'),
 ('POG', 'THM'),
 ('POG', 'PRG'),
 ('POG', 'RUG'),
 ('POG', 'RUM'),
 ('POG', 'COG'),
 ('POG', 'COM'),
 ('THG', 'THM'),
 ('THG', 'PRG'),
 ('THG', 'RUG'),
 ('THG', 'RUM'),
 ('THG', 'COG'),
 ('THG', 'COM'),
 ('THM', 'PRG'),
 ('THM', 'RUG'),
 ('THM', 'RUM'),
 ('THM', 'COG'),
 ('THM', 'COM'),
 ('PRG', 'RUG'),
 ('PRG', 'RUM'),
 ('PRG', 'COG'),
 ('PRG', 'COM'),
 ('RUG', 'COG'),
 ('RUG', 'COM'),
 ('RUM', 'COG'),
 ('RUM', 'COM'),
 ('POG',),
 ('THG',),
 ('THM',),
 ('PRG',),
 ('RUG',),
 ('RUM',),
 ('COG',),
 ('COM',)]

Now to get a list of all possible states given a certain state. Notes:

- Since I'm storing floors as a tuple of tuples so I can hash states into a dict, first up I need to convert floors into a list
- use sets to add and remove items from a floor

In [14]:
def get_states(st):
    """takes in a state and returns all possible states"""
    
    elavator = st.elavator
    floors = list(set(floor) for floor in st.floors)
    
    floors_to_modify = [i for i in range(elavator-1, elavator+2) if (i>=0 and i is not elavator and i<=3)]
    
    for fl in floors_to_modify[::-1]: # to prioritize going up
        for c in get_combos(floors[elavator]):
            new_floor = floors[fl] | set(c) # add new items to the floor they are moving to
            elavator_floor = floors[elavator] - set(c) # remove the items from the elevator floor
            
            if is_legal_floor(new_floor) and is_legal_floor(elavator_floor):
                new_floors = list(tuple(sorted(floor)) for floor in st.floors)
                new_floors[fl] = tuple(new_floor) # updating the floor items are moving to
                new_floors[elavator] = tuple(elavator_floor) # changing the elavotor floor
                
                yield state(elavator=fl, steps=st.steps+1, floors=tuple(new_floors))
    
[i for i in get_states(start_state)]

[State(elavator=1, steps=1, floors=(('COG', 'RUG', 'THG', 'POG', 'COM', 'PRG'), ('RUM', 'THM', 'PRM', 'POM'), (), ())),
 State(elavator=1, steps=1, floors=(('COG', 'RUG', 'THG', 'THM', 'POG', 'PRG'), ('RUM', 'COM', 'PRM', 'POM'), (), ())),
 State(elavator=1, steps=1, floors=(('RUM', 'COG', 'RUG', 'THG', 'POG', 'PRG'), ('THM', 'PRM', 'COM', 'POM'), (), ())),
 State(elavator=1, steps=1, floors=(('RUM', 'COG', 'RUG', 'THG', 'THM', 'COM'), ('PRG', 'PRM', 'POG', 'POM'), (), ())),
 State(elavator=1, steps=1, floors=(('COG', 'RUG', 'THG', 'THM', 'POG', 'COM', 'PRG'), ('RUM', 'PRM', 'POM'), (), ())),
 State(elavator=1, steps=1, floors=(('RUM', 'COG', 'RUG', 'THG', 'POG', 'COM', 'PRG'), ('THM', 'PRM', 'POM'), (), ())),
 State(elavator=1, steps=1, floors=(('RUM', 'COG', 'RUG', 'THG', 'THM', 'POG', 'PRG'), ('COM', 'PRM', 'POM'), (), ()))]

So that seems to be working properly, now to solve this. Notes:

- I was using a python list as a stack of states to check, but turns out this takes O(n) time to pop the first node, so switched to [deque](https://docs.python.org/3/library/collections.html#collections.deque)

In [15]:
def solve(st):
    """takes in a start state and returns first state which solves the problem"""
    stack = deque()
    stack.append(st)

    seen = set()

    st = stack.popleft()

    while not reached_goal(st):
        if st not in seen:
            stack.extend([i for i in get_states(st) if i not in seen]) # add all new unseen states to the stack
        seen.add(st) # have now processed this state
        
        st = stack.popleft() # pop the first in state

    return st

%time solve(test_state)

CPU times: user 32 ms, sys: 0 ns, total: 32 ms
Wall time: 33 ms


State(elavator=3, steps=11, floors=((), (), (), ('LIG', 'LIM', 'HYG', 'HYM')))

In [ ]:
%time solve(start_state)

`47` is the right ans for my puzzle input, but 5 min is a long time!

# --- Part Two ---

You step into the cleanroom separating the lobby from the isolated area and put on the hazmat suit.

Upon entering the isolated containment area, however, you notice some extra parts on the first floor that weren't listed on the record outside:

- An elerium generator.
- An elerium-compatible microchip.
- A dilithium generator.
- A dilithium-compatible microchip.

These work just like the other generators and microchips. You'll have to get them up to assembly as well.

What is the minimum number of steps required to bring all of the objects, including these four new ones, to the fourth floor?

---

So this is exactly the same as above, but with a much bigger search space. So my 5 min solution is not going to cut the mustard here...

First up, the new starting state is:

In [16]:
one = ("POG", "THG", "THM", "PRG", "RUG", "RUM", "COG", "COM", "ELG", "ELM", "DIG", "DIM")
two = ("POM", "PRM")
three, four = (), ()

second_puzzle_state = state(elavator=0,steps=0, floors=(one, two, three, four))
second_puzzle_state

State(elavator=0, steps=0, floors=(('POG', 'THG', 'THM', 'PRG', 'RUG', 'RUM', 'COG', 'COM', 'ELG', 'ELM', 'DIG', 'DIM'), ('POM', 'PRM'), (), ()))

This is much longer than the first puzzle, so I need to speed up my solution.

- **filter out states which are the same for this problem** all matched chip and gen are interchangeble for other matched chip/gen pairs in this problem.
- implement a better solution like [a star](https://www.redblobgames.com/pathfinding/a-star/introduction.html) - not done yet

So first up I need a way to check if a state is similar to another, which I do by the following really horrible hash state which:

- removes the pairs from the list of floors and saves that in a seperate pairs tuple
- sorts the floors and puts them in a list
- returns a string

BUT the `make_hash` func is doing a lot of steps. simplify!

In [17]:
hash_state = namedtuple("HashState", ["elavator", "steps", "pairs", "floors"])
from functools import lru_cache

@lru_cache(maxsize=2**16)
def make_hash(st):
    """takes in a state and returns its hashed state which treats chip/gen pairs equally"""
    floors = [list(fl) for fl in st.floors]
    num_pairs = [0 for _ in range(4)]
    
    for i, fl in enumerate(floors):
        # find num of matching gen/chip pair
        gens = {i[:2] for i in fl if i[-1] == "G"}
        chips = {i[:2] for i in fl if i[-1] == "M"}
        pairs = gens & chips
        # remove the pairs from the list of items
        floors[i] = "".join(sorted([i for i in fl if i[:-1] not in pairs]))
        num_pairs[i] = len(pairs)
    
    return str(st.elavator) + "_" + str(st.steps) +"_" \
    + "_".join([str(i) for i in num_pairs]) \
    + "_".join(floors)

%time make_hash(start_state)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 39.8 µs


'0_0_3_0_0_0POGPRG_POMPRM__'

Another optimization step is to filter out some states, specifically:

- once first floor is empty, don't move items back down to it 
- TODO: when moving items down, if can move 1 item down, then don't move two items down. This is becuase in situations we have to move 1 item down, then move 2 items back up, but there is no cause where moving a matched chip/gen pair down a floor helps.

In [18]:
def filter_states(states):
    """only returns states with first floor empty. if doesn't find such a state returns all
    the states passed in.
    TODO: add more filters"""
    first_floor_empty = [st for st in states if st.floors[0] == ()]
    if first_floor_empty != []:
        return first_floor_empty
    return states
    
filter_states([start_state, second_puzzle_state, solved_state])

[State(elavator=3, steps=4, floors=((), (), (), (1, 2)))]

In [19]:
def get_filtered_states(st):
    """takes in a state and returns the states which progresses towards our goals"""
    
    #TODO: modify this func so it deals with up and down floors seperately
    # if can move one item down then only return those states, only deal with two down if no one items
    
    elavator = st.elavator
    floors = list(set(floor) for floor in st.floors)
    
    floors_to_modify = [i for i in range(elavator-1, elavator+2) if (i>=0 and i is not elavator and i<=3)]
    
    for fl in floors_to_modify[::-1]: # to prioritize going up
        for c in get_combos(floors[elavator]):
            new_floor = floors[fl] | set(c) # add new items to the floor they are moving to
            elavator_floor = floors[elavator] - set(c) # remove the items from the elevator floor
            
            if is_legal_floor(new_floor) and is_legal_floor(elavator_floor):
                new_floors = list(tuple(sorted(floor)) for floor in st.floors)
                new_floors[fl] = tuple(new_floor) # updating the floor items are moving to
                new_floors[elavator] = tuple(elavator_floor) # changing the elavotor floor
                
                yield state(elavator=fl, steps=st.steps+1, floors=tuple(new_floors))
    
[i for i in get_states(start_state)]

[State(elavator=1, steps=1, floors=(('COG', 'RUG', 'THG', 'POG', 'COM', 'PRG'), ('RUM', 'THM', 'PRM', 'POM'), (), ())),
 State(elavator=1, steps=1, floors=(('COG', 'RUG', 'THG', 'THM', 'POG', 'PRG'), ('RUM', 'COM', 'PRM', 'POM'), (), ())),
 State(elavator=1, steps=1, floors=(('RUM', 'COG', 'RUG', 'THG', 'POG', 'PRG'), ('THM', 'PRM', 'COM', 'POM'), (), ())),
 State(elavator=1, steps=1, floors=(('RUM', 'COG', 'RUG', 'THG', 'THM', 'COM'), ('PRG', 'PRM', 'POG', 'POM'), (), ())),
 State(elavator=1, steps=1, floors=(('COG', 'RUG', 'THG', 'THM', 'POG', 'COM', 'PRG'), ('RUM', 'PRM', 'POM'), (), ())),
 State(elavator=1, steps=1, floors=(('RUM', 'COG', 'RUG', 'THG', 'POG', 'COM', 'PRG'), ('THM', 'PRM', 'POM'), (), ())),
 State(elavator=1, steps=1, floors=(('RUM', 'COG', 'RUG', 'THG', 'THM', 'POG', 'PRG'), ('COM', 'PRM', 'POM'), (), ()))]

In [20]:
def solve_faster(st):
    """takes in a start state and returns first state which solves the problem"""
    stack = deque()
    stack.append(st)

    seen = set()

    st = stack.popleft()
    
    i = 0 # for the prints
    
    while not reached_goal(st):
        hashed_state = make_hash(st)
        if hashed_state not in seen:
            new_states = filter_states([i for i in get_states(st) if make_hash(i) not in seen])
            stack.extend(new_states) # add all new unseen states to the stack
            
        seen.add(hashed_state) # have now processed this state
        
        st = stack.popleft() # pop the first in state
        
        i += 1
        if i % 1000000 == 0:
            print(f"on loop {i:14,d} looking at:")
            print(st)
    
    return st

%time solve_faster(test_state)

CPU times: user 40 ms, sys: 0 ns, total: 40 ms
Wall time: 38.6 ms


State(elavator=3, steps=11, floors=((), (), (), ('LIG', 'LIM', 'HYG', 'HYM')))

In [21]:
%time solve_faster(start_state)

on loop        900,000 looking at:
State(elavator=0, steps=28, floors=(('THG', 'THM', 'RUG'), ('PRG', 'COG', 'POG'), ('PRM', 'RUM'), ('COM', 'POM')))
on loop      1,800,000 looking at:
State(elavator=0, steps=34, floors=(('THG', 'RUM', 'COG', 'RUG', 'PRG'), (), ('POG', 'POM'), ('COM', 'PRM', 'THM')))
on loop      2,700,000 looking at:
State(elavator=1, steps=39, floors=((), ('THG', 'THM', 'COG', 'PRM', 'POG', 'RUG', 'COM', 'PRG', 'POM'), (), ('RUM',)))
on loop      3,600,000 looking at:
State(elavator=0, steps=44, floors=(('THG', 'RUM', 'PRM', 'POG', 'COG', 'RUG', 'PRG', 'POM'), (), (), ('COM', 'THM')))
CPU times: user 2min 54s, sys: 232 ms, total: 2min 54s
Wall time: 2min 54s


State(elavator=3, steps=47, floors=((), (), (), ('RUM', 'COG', 'RUG', 'THG', 'THM', 'PRM', 'POG', 'COM', 'PRG', 'POM')))

So `solve_faster` is only 2x faster than my original solver at ~3min - that is still very slow, the reddit hints suggest I should be able to solve this in under a min. So there is no way it can solve the second puzzle part.

In [ ]:
solve_faster(second_puzzle_state)